In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [ ]:
class ConvBlock(nn.Module):
    def __init__(self, in_channels, out_channels, num_conv_layers=2):
        super(ConvBlock, self).__init__()
        self.num_conv_layers = num_conv_layers
        self.conv_layers = nn.ModuleList()

        # Create the specified number of convolutional layers
        for _ in range(self.num_conv_layers):
            self.conv_layers.append(
                nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1)
            )
            in_channels = out_channels  # Update in_channels after each layer
        
        self.batch_norm = nn.BatchNorm2d(out_channels)

    def forward(self, x):
        for conv in self.conv_layers:
            x = F.relu(self.batch_norm(conv(x)))
        return x

In [ ]:
class DownBlock(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(DownBlock, self).__init__()
        self.conv_block = ConvBlock(in_channels, out_channels)
        self.pool = nn.MaxPool2d(2)
    
    def forward(self, x):
        x = self.conv_block(x)
        p = self.pool(x)
        return x, p

In [ ]:
class UpBlock(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(UpBlock, self).__init__()
        self.upconv = nn.ConvTranspose2d(in_channels, out_channels, kernel_size=2, stride=2)
        self.conv_block = ConvBlock(in_channels, out_channels)
    
    def forward(self, x, skip):
        x = self.upconv(x)
        x = torch.cat([x, skip], dim=1)
        x = self.conv_block(x)
        return x

In [ ]:
class UNetPlusPlus(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(UNetPlusPlus, self).__init__()
        
        # Encoder (Contracting Path)
        self.enc1 = DownBlock(in_channels, 64)
        self.enc2 = DownBlock(64, 128)
        self.enc3 = DownBlock(128, 256)
        self.enc4 = DownBlock(256, 512)
        
        # Bottleneck
        self.bottleneck = ConvBlock(512, 1024)
        
        # Decoder (Expansive Path)
        self.up4 = UpBlock(1024, 512)
        self.up3 = UpBlock(512, 256)
        self.up2 = UpBlock(256, 128)
        self.up1 = UpBlock(128, 64)
        
        # Final convolution for segmentation output
        self.final_conv = nn.Conv2d(64, out_channels, kernel_size=1)
        
        # Nested skip pathways (the core modification in U-Net++)
        self.nested_skip1 = ConvBlock(64, out_channels, num_conv_layers=3)  # 3 conv layers
        self.nested_skip2 = ConvBlock(128, out_channels, num_conv_layers=2)  # 2 conv layers
        self.nested_skip3 = ConvBlock(256, out_channels, num_conv_layers=2)  # 2 conv layers
        self.nested_skip4 = ConvBlock(512, out_channels, num_conv_layers=1)  # 1 conv layer

    def forward(self, x):
        # Encoder
        enc1, p1 = self.enc1(x)
        enc2, p2 = self.enc2(p1)
        enc3, p3 = self.enc3(p2)
        enc4, p4 = self.enc4(p3)
        
        # Bottleneck
        bottleneck = self.bottleneck(p4)
        
        # Decoder with nested skip connections
        up4 = self.up4(bottleneck, enc4)
        up3 = self.up3(up4, enc3)
        up2 = self.up2(up3, enc2)
        up1 = self.up1(up2, enc1)

        # Final segmentation output
        segmentation_output = self.final_conv(up1)

        # Nested skip connections outputs (dense skip paths)
        nested_skip1_out = self.nested_skip1(enc1)
        nested_skip2_out = self.nested_skip2(enc2)
        nested_skip3_out = self.nested_skip3(enc3)
        nested_skip4_out = self.nested_skip4(enc4)
        
        # Combine nested skip outputs with segmentation output
        final_output = segmentation_output + nested_skip1_out + nested_skip2_out + nested_skip3_out + nested_skip4_out

        return final_output